In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re


### Read CSV

In [2]:
url_hosts = pd.read_csv("distinct.csv")
url_hosts = url_hosts.drop_duplicates()
print("Loaded {} unique URLs".format(len(url_hosts)))
#url_hosts = url_hosts.iloc[:50]

Loaded 4669 unique URLs


### Scrape URL - Title/Body

In [7]:
def read_page(url_path):   
    global page_raw
    global page_soup
    page_raw = get("http://"+url_path).text
    page_soup = BeautifulSoup(page_raw, 'lxml')
    try:
        title = page_soup.find("title").text
    except:
        title = "N/A"
    try:
        body = page_soup.find("body")
    except:
        body = "N/A"
        
    return (title, body)

In [8]:
read_page("lowes.com")

("Lowe's Home Improvement", <body>
 <a class="ada screen-reader-only screen-reader-only-focusable" href="#mainContent">Skip to main content</a>
 <a class="ada screen-reader-only screen-reader-only-focusable" href="#mainNavigation">Skip to main navigation</a>
 <div class="home global page" id="wrapper">
 <div id="main">
 <!-- ESI include /globalassets/masthead at 2019-09-13T5:54:45.508-0400 -->
 <div class="mpulseloader parbase">
 </div>
 <div class="masthead js-masthead" data-disabled-mode="true">
 <!-- masthead desktop -->
 <header class="head js-gauge-header">
 <div class="top-bar js-top-bar animated fastest">
 <div class="fluid-grid-container">
 <div class="grid-container">
 <div class="grid-100 tablet-grid-100 grid-parent">
 <div class="storeindicator">
 <script id="CurrentStoreView" type="text/mustache">
 	{{#if isSchedulePage}}
 		<div class="store-hours selected-store">
 		{{storeName}}
 			<span class="font-weight-normal black">
 				{{#if isHoliday}}
 					{{#if isHolidayOpenT

In [ ]:
title_list = []
body_list = []
for ix in tqdm(range(len(url_hosts['URL_HOST']))):
    url = url_hosts.iloc[ix]['URL_HOST']
    try:
        page_data = read_page("http://"+url)
    except:
        title_list.append("N/A")
        body_list.append("N/A")
    title_list.append(page_data[0])
    body_list.append(page_data[1])
    

url_data = url_hosts.copy()
url_data['title'] = title_list
url_data['body'] = body_list

 13%|█████████████▍                                                                                            | 593/4669 [05:42<40:17,  1.69it/s]

In [16]:
pd.read_csv("s3://ione-datascience-3be7-us-east-1/ione.datascience/projects/contextual/01-urls/distinct.csv")

PermissionError: Access Denied

### Save to CSV

In [ ]:
url_data.to_csv("scraped_urls.csv")